In [ ]:
# default_exp auth

# auth

> python sdk for interacting with aws backend

In [5]:
#hide
from nbdev.showdoc import *

In [17]:
#export
from nicHelper.wrappers import add_method
from lambdasdk.lambdasdk import Lambda
from awsSchema.apigateway import Event, Response
from nicHelper.dictUtil import printDict
from nicHelper.exception import errorString
from dataclasses import dataclass
from dataclasses_json import dataclass_json

In [56]:
#export
class FunctionName:
  def __init__(self, branch, appName):
    self.branch = branch; self.appName=appName
  def getName(self,function):
    return f'{self.appName}-{self.branch}-{function}'
    
class AuthSdk:
  def __init__(
    self,
    user = None,
    pw = None,
    sessionToken = None,
    region = 'ap-southeast-1',
    branch = 'dev-blank',
    appName = 'villa-employee'
    ):
    self.lambda_ = Lambda(user = user, pw = pw, sessionToken= sessionToken, region = region)
    self.functionName = FunctionName(branch, appName)
    
  def generalInvoke(self, body={}, function= 'unauth'):
    functionName = self.functionName.getName(function)
    event = Event.getInput(body=body)
    response = self.lambda_.invoke(functionName=functionName, input=event)
    try: return Response.parseBody(response)
    except: return response
  

In [57]:
#hide
sdk = AuthSdk()
# sdk.lambda_.invoke(functionName = 'test', input = {'test':'test'})

In [60]:
#export
@dataclass_json
@dataclass
class CreateCashier:
  user: str
  phone: str
  pw: str
  name: str
  cashierId: str
    
@add_method(AuthSdk)
def createCashier(self,data={}, function= 'createCashier'):
  payload = CreateCashier(**data).to_dict()
  return self.generalInvoke(body=payload,function=function)

In [61]:
sdk.createCashier({
  'user': 'nic1',
  'phone': '+66816684442',
  'pw': '12345678',
  'name': 'nic',
  'cashierId': '001'
})

{'error': {'success': False, 'error': 'user exists'}}

In [73]:
#export
@dataclass_json
@dataclass
class Auth:
  user: str
  password: str
@add_method(AuthSdk)
def auth(self,user,pw, function= 'auth'):
  payload = Auth(user=user, password=pw).to_dict()
  return self.generalInvoke(body=payload,function=function)

In [74]:
result = AuthSdk().auth(user='nic1',pw='12345678')
printDict(result)

AccessKeyId : ASIAVX4Z5T
SecretKey : 1FU7+fVGLL
SessionToken : IQoJb3JpZ2
Expiration : 1606978696.0
AccessToken : eyJraWQiOi
ExpiresIn : 3600
TokenType : Bearer
RefreshToken : eyJjdHkiOi
IdToken : eyJraWQiOi
NewDeviceMetadata
 DeviceKey : ap-southea
 DeviceGroupKey : -lt0cpemm
userInfo
 Username : nic1
 UserCreateDate : 1606963203.881
 UserLastModifiedDate : 1606963203.881
 Enabled : True
 UserStatus : CONFIRMED
 ResponseMetadata
  RequestId : 1cfc5822-9
  HTTPStatusCode : 200
  HTTPHeaders
   date : Thu, 03 De
   content-type : applicatio
   content-length : 339
   connection : keep-alive
   x-amzn-requestid : 1cfc5822-9
  RetryAttempts : 0
kwUserAttrib
 sub : d7ca5125-6
 name : nic
 phone_number_verified : true
 phone_number : +668166844


In [75]:
# result

In [82]:
#export
@dataclass_json
@dataclass
class Confirm:
  user: str
  code: str
@add_method(AuthSdk)
def confirm(self,user:str, code:str, function:str='confirm', **kwargs):
  payload = Confirm(user=user, code=code).to_dict()
  return self.generalInvoke(body=payload, function=function)

In [84]:
result=AuthSdk().confirm('nic1', '123')
printDict(result)

error
 pickledb64string : gASVTQUAAA
 error : confirmati


In [95]:
#export
@dataclass_json
@dataclass
class GetProfile:
  user: str
    
@add_method(AuthSdk)
def getProfile(self,user:str, function:str='getProfile', **kwargs):
  payload = GetProfile(user=user).to_dict()
  return self.generalInvoke(body=payload, function=function)

In [104]:
result=sdk.getProfile('nic1')
printDict(result)

Username : nic1
UserAttributes
 sub : d7ca5125-6
 name : nic
 custom:cashierCode : 1234
 phone_number_verified : true
 phone_number : +668166844
UserCreateDate : 1606963203.881
UserLastModifiedDate : 1606977796.225
Enabled : True
UserStatus : CONFIRMED
ResponseMetadata
 RequestId : 263aff07-3
 HTTPStatusCode : 200
 HTTPHeaders
  date : Thu, 03 De
  content-type : applicatio
  content-length : 384
  connection : keep-alive
  x-amzn-requestid : 263aff07-3
 RetryAttempts : 0


In [106]:
#export
@dataclass_json
@dataclass
class UpdateProfile:
  user: str
  attributes: dict
    
@add_method(AuthSdk)
def updateProfile(self,user:str, attributes:dict, function:str='updateProfile', **kwargs):
  payload = UpdateProfile(user=user, attributes=attributes).to_dict()
  return self.generalInvoke(body=payload, function=function)

In [109]:
result = sdk.updateProfile(**{
  'user': 'nic1',
  'attributes': {
    'custom:cashierCode': '1234'
  }
})
printDict(result)

Username : nic1
UserAttributes
 sub : d7ca5125-6
 name : nic
 custom:cashierCode : 1234
 phone_number_verified : true
 phone_number : +668166844
UserCreateDate : 1606963203.881
UserLastModifiedDate : 1606978149.418
Enabled : True
UserStatus : CONFIRMED
ResponseMetadata
 RequestId : 442f9a54-2
 HTTPStatusCode : 200
 HTTPHeaders
  date : Thu, 03 De
  content-type : applicatio
  content-length : 384
  connection : keep-alive
  x-amzn-requestid : 442f9a54-2
 RetryAttempts : 0


In [111]:
#export
@dataclass_json
@dataclass
class SetPassword:
  user: str
  pw: str
    
@add_method(AuthSdk)
def setPassword(self,user:str, pw:str, function:str='setPassword', **kwargs):
  payload = SetPassword(user=user, pw=pw).to_dict()
  return self.generalInvoke(body=payload, function=function)

In [112]:
sdk.setPassword(user='nic1',pw='12345678')

{'ResponseMetadata': {'RequestId': '323fd8b0-0aed-479c-9be5-588089a2970d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 03 Dec 2020 06:50:53 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '323fd8b0-0aed-479c-9be5-588089a2970d'},
  'RetryAttempts': 0}}

In [117]:
@add_method(AuthSdk)
def unauth(self, function:str='unauth', **kwargs):
  return self.generalInvoke(body={}, function=function)

In [118]:
result = sdk.unauth()
printDict(result)

AccessKeyId : ASIAVX4Z5T
SecretKey : wW377g9c54
SessionToken : IQoJb3JpZ2
Expiration : 1606981946.0
